
# FitResult

In this tutorial, we will explore the `FitResult`of zfit. Specifically, we will examine the error methods hesse and errors as well as attributes like info, valid etc. We will also provide an example with weighted data to demonstrate how FitResult works with weighted datasets.



We will start out by creating a simple gaussian model and sampling some data from it. We will then fit the data with the same model and explore the `FitResult`.

In [ ]:
import numpy as np
import zfit
import zfit.z.numpy as znp

In [ ]:
obs = zfit.Space('x', 0, 10)
mu = zfit.Parameter('mu', 5, 0, 10)
sigma = zfit.Parameter('sigma', 1, 0, 10)
nsig = zfit.Parameter('nsig', 1000, 0, 10000)
gauss = zfit.pdf.Gauss(obs=obs, mu=mu, sigma=sigma,extended=nsig)
data = gauss.sample()
print(f"The sampled data (poisson fluctuated) has {data.nevents} events.")

We use an extended likelihood to fit the data.

In [ ]:
nll = zfit.loss.ExtendedUnbinnedNLL(model=gauss, data=data)
minimizer = zfit.minimize.Minuit()
result = minimizer.minimize(nll)

Simply printing the result will give you a beautified overview of the fit result.

In [ ]:
print(result)

## What happened

First and foremost, the FitResult contains all the information about what happened with the minimization, most notably the `loss` that was minimized, the `minimizer` that was used and the `params` that were fitted (the latter has a beautified presentation).

In [ ]:
print(f"""
loss: {result.loss}
minimizer: {result.minimizer}
params: {result.params}
""")

### params

`params` contains all the information of the parameters that was ever added to them. This includes the output of uncertainty methods, limits and much more.
The actual content looks like this:

In [ ]:
print(f"params raw: {repr(result.params)}")

The `FitResult` has a lot of attributes and methods. We will now explore some of them.


All the displayed information can be accessed via the attributes of the `FitResult` object, namely
- valid: whether the fit converged and is in general valid
- converged: whether the fit converged
- param at limit: whether any parameter is at its limit (approximate, hard to estimate)
- edm: estimated distance to minimum
- fmin: the minimum of the function, i.e. the negative log likelihood
- values: the parameter values at the minimum in an array-like object

In [ ]:
print(f"""
valid: {result.valid}
converged: {result.converged}
param at limit: {result.params_at_limit}
edm: {result.edm}
fmin: {result.fmin}
optimal values: {result.values}
""")

## Error methods

There are two main ways to estimate the uncertainties: Either using a profiling method that varies the parameters one by one and finds
the point of 1 sigma (or the specified n sigma), resulting in asymmetric errors, or using a matrix inversion method that calculates
an approximation of the former by using a second derivative matrix.

The first method is called `errors` and the second `hesse`. Both methods are available in the `FitResult` object.

### Pitfall weights

For weighted likelihoods, the `errors` method will not report the correct uncertainties. Instead, `hesse` should be used
as it will, by default, calculate the asymptotic correct approximations for weights as we will see a few lines below.

### Arguments

Both methods take some common arguments:
- `params`: the parameters to calculate the errors for. If `None`, all parameters will be used. (this can be expensive!)
- `name`: the name of the new result. If `None`, the name will be chosen automatically.
- `cl`: the confidence level for the errors. The default is 0.68, which corresponds to 1 sigma.
- `method`: the method to use. The default is `None` which will use the default method of the uncertainty estimator.

In [ ]:
errors, new_result = result.errors(name="errors")
print(f"New result: {new_result}")
print(result)

The uncertainties are added to the fit result. The `new_result` is usually `None` but in case a new minimum was found, it will be returned
as the new result. In this case, the old result will be rendered invalid.

There are currently two implementations, the minos method from `iminuit` (as `minuit_minos`) and a completely independent implementation
(`zfit_errors`).

### More information

To find more information about the uncertainty estimation, the return value can be inspected. This is though also automatically added to the result
to each parameter. Looking again at the raw `params` attribute, we find that all the information is there:

_Note: this part is still under WIP and future plans are to standardize these attributes as well. Any ideas or inputs are very welcome!_

In [ ]:
print(f"params raw: {repr(result.params)}")

In [ ]:
errors2, _ = result.errors(name="zfit_unc", method="zfit_errors")
print(result)

As we see, they both agree well. We can also change the confidence level to 0.95, which corresponds to 2 sigma and recalculate the errors.

In [ ]:
errors3, _ = result.errors(name="zfit_2sigma", method="zfit_errors", cl=0.95)
print(result)

### Hesse

The hesse method approximates the errors by calculating the second derivative matrix of the function and inverting it.
As for `errors` there are two implementations, one from `iminuit` (`minuit_hesse`) and one from `zfit` (`hesse_np`).

Additionally, the `hesse` has a third option, `approx`: this is the approximation of the hessian estimated by the minimizer
during the minimization procedure. This however *can* be `None`! Also, the accuracy can be low, especially if the
fit converged rapidly.

In [ ]:
hesse = result.hesse(name="h minuit", method="minuit_hesse", cl=0.95)  # can also take the cl argument
hesse2 = result.hesse(name="h zfit", method="hesse_np")
hesse3 = result.hesse(name="h approx", method="approx")
print(result)

Internally, zfit uses by default a numerical approximation of the hessian, which is usually sufficient and good for one-time use.
However, if you want to use the hessian for multiple fits, it is recommended to force it to use the exact gradient provided by the
backend. To make sure one or the other is used, you can set `zfit.run.set_autograd_mode(False)` or `zfit.run.set_autograd_mode(True)`.

In [ ]:
with zfit.run.set_autograd_mode(True):
    hesse4 = result.hesse(name="h autograd", method="hesse_np")
print(result)

## Weighted uncertainties

A weighted likelihood is technically not a likelihood anymore and the errors are not calculated correctly. However, the hesse method
can be corrected for weights, which is done automatically as soon as the dataset is weighted.

The method used is the `asymptotically correct` yet computationally expensive method described in [Parameter uncertainties in weighted unbinned maximum likelihood fits](https://link.springer.com/article/10.1140/epjc/s10052-022-10254-8).

The computation involves the jacobian of each event that can be expensive to compute. Again, zfit offers the possibility to use the
autograd or the numerical jacobian.

In [ ]:
weighted_data = zfit.Data.from_tensor(obs=obs, tensor=data.value(), weights=znp.random.uniform(0.1, 5, size=(data.nevents,)))
weighted_nll = zfit.loss.UnbinnedNLL(model=gauss, data=weighted_data)
weighted_result = minimizer.minimize(weighted_nll)

In [ ]:
weighted_result.errors(name="errors")

In [ ]:
with zfit.run.set_autograd_mode(True):
    weighted_result.hesse(name="hesse autograd")
    weighted_result.hesse(name="hesse autograd np", method="hesse_np")

In [ ]:
with zfit.run.set_autograd_mode(False):
    weighted_result.hesse(name="hesse numeric")
    weighted_result.hesse(name="hesse numeric np", method="hesse_np")

In [ ]:
print(weighted_result)  # FIXME: the errors are not correct for the nsig

As we can see, the errors are underestimated for the nuisance parameters using the minos method while the hesse method is correct.

### Standardized minimizer information

Some of the minimizers collect information about the loss during the minimization process, such as an approximation of the hessian, inverse hessian, gradient etc. They can be retrieved via `approx`, note however that they can be `None`.

`hessian` and `inv_hessian` have an `invert` argument: if True and only one of the two is available, the other one will be inverted to obtain the request.


In [ ]:
print(f"Approx gradient: {result.approx.gradient()}")  # gradient approx not available in iminuit
print(f"Approx hessian (no invert): {result.approx.hessian(invert=False)}")  # hessian approximation is also not available
print(f"Approx inverse hessian: {result.approx.inv_hessian(invert=False)}")  # inv_hessian is available
print(f"Approx hessian (can invert): {result.approx.hessian(invert=True)}")  # allowing the invert now inverts the inv_hessian

### info
The information returned by the minimizer. CAREFUL! This is a dictionary and can be different for different minimizers. The standardized keys can always be accessed in other ways, such as the approximations of the hessian, the covariance matrix etc.

In [ ]:
result.info.keys()

This can be helpful if underlying information from a specific minimizer should be retrieved. For example, the `original` key contains the original result from the minimizer while "minuit" is the actual `iminuit` minimizer that was used.

In [ ]:
result.info.get("original", f"Not available for the minimizer: {result.minimizer}")

In [ ]:
result.info.get("minuit", "Not available, not iminuit used in minimization?")

### Finding problems

If the fit failed for some reason, `valid` may be False. To find the actual reason, `message` should be human-readable information about what went wrong. If everything went well, the message will be empty

In [ ]:
result.message